In [291]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as opt
from scipy.special import expit, logit
import pandas as pd

## Expected
- t = True incidence of left-handedness
- T = Probability matrix of P(child's hand | parents' hand)

In [292]:
def compute_T(ρ, α, β):
    """Generate T - column DD in Table 1
    
    From Appendix 3:    
    T is a 3 x 2 matrix of p(Ht | Ht x Ht) entries - 
    probability of a truly H child from a mating where parents are truly H
    """
    return np.array([
        # R           L
        [0.5 + ρ + α, 1 - (0.5 + ρ + α) ],# RxR
        [0.5 + ρ + β, 1 - (0.5 + ρ + β) ],# RxL
        [0.5 + ρ - α, 1 - (0.5 + ρ - α) ] # LxL
    ])

In [293]:
# plot comput_T in a nice and tidy table:

def table_T(ρ, α, β):
    table_T = pd.DataFrame(data = compute_T(ρ, α, β), index = ['RxR','RxL','LxL'], columns = ['Ex_Right', 'Ex_Left'])
    return table_T

In [296]:
#Joint arry of t and T - > expected measures

def expected(ρ, α, β):
    return compute_t(ρ, α, β), T_table(ρ, α, β)

In [298]:
def compute_t(ρ, α, β):
   #"""Generate t - eq .3 - the true incidence of left-handedness
    
#     From main text:
#     F_DR is the final equilibrium frequency of right-handers with allele D fixed
#     We need F_DL = 1 - F_DR"""

    if β == 0:        
        FDR = (1 - 2 * α + 2 * ρ) / (2 * (1 - 2 * α)) # eq 3a
    else:
        Δ = 4 * α * α - 4 * α + 4 * β * β + 1 + 8 * β * ρ
        FDR = (2 * α + 2 * β - 1 + np.sqrt(Δ)) / (4 * β) # eq 3    
    return print('P(DR) = ', FDR,'\nP(DL) =' , 1 - FDR)

## Observed

In [373]:
# Observed frequancy of P(offsprin|parents mating) in a given sample D:

def observed_T(D):
    
    RxR = D[:,0:2].sum()
    RxL = D[:,2:4].sum()
    LxL = D[:,4:6].sum()
    
    D_obs = pd.DataFrame( data = [
        #   R                     L 
        [D[:,0].sum()/RxR , D[:,1].sum()/RxR], #RxR
        [D[:,2].sum()/RxL , D[:,3].sum()/RxL], #RxL
        [D[:,4].sum()/LxL , D[:,5].sum()/LxL], #LxL
        ], index = ['RxR', 'RxL', 'LxL'],  columns = ['Obs_R', 'Obs_L'])
    
    return D_obs

In [379]:
# Observed total frequancy of right and left handers in a given sample D:

def observed_total(D):
    obs_total=[
        [(3*D[:,0] + 2*D[:,1] + 2*D[:,2] + D[:,3] + D[:,4]).sum()],  #Right
        [(D[:,1] + D[:,2] + 2*D[:,3] + 2*D[:,4] + 3*D[:,5]).sum()]  #Left  
    ]
    obs_total = np.array(obs_total)
    obs_total = pd.DataFrame(data= obs_total/obs_total.sum(), index= ['Left', 'Right'], columns = ['Total sample'])
    return obs_total

In [297]:
#Joint arry of sample frequencies:
def observed(D):
    return observed_total(D), observed_T(D)

## Validate

In [299]:
def validate(θ):
    if len(θ)==3:
        ρ, α, β = θ
    elif len(θ)==2:
        ρ, α = θ
        β = 0
    else:
        raise ValueError("Length of θ is {}".format(len(θ)))
        
    if not 0 <= ρ <= 1 or not 0 <= α <= 1 or not 0 <= β <= 1 or ρ + α > 0.5 or ρ - α > 0.5 or ρ + β > 0.5:
        return False
    return True

## Parameters

In [300]:
θ_1995 = np.array([0.267, 0.148, 0.012])
θ_19952d = np.array([0.277, 0.138, 0])

In [300]:
θ_Laland_new =  np.array([0.320, 0.157, 0.000]) # bayseian inference
θ_19952d_new = np.array([0.321, 0.156, 0]) # bayseian inference

# Results

הערה: הטבלה של הצפי של ילדים כתלות בהורים לא סוכמת ל1 בטורים אלא רק בשורות.....אז צריך להתאים את הטבלאות של הדאטה הנצפה לזה. 

## Laland 1995 

### Data

In [361]:
#    R|RxR  L|RxR  R|RxL  L|RxL  R|LxL  L|LxL

Laland = [
    [841,  115,   113,   54,    1,    7],   # Ramaley 1913, paper says it does not fit
    [6917, 308,   411,   53,    18,    7],  # Chamberlain 1928 
    [1842, 151,   140,   34,    5,     6],  # Rife 1940
    [140,  34,    33,    20,    8,     2],  # Merrell 1975
    [6206, 669,   471,   125,   5,     1],  # Annett 1973
    [154,  11,    31,    9,     0,     0],  # Ferronato et al. 1947
    [232,  17,    41,    7,     3,     1],  # Mascie-Taylor (unpub, in McManus 1985)
    [1060, 144,   122,   46,    3,     4],  # Chaurasia & Groswani (unpub, in McManus 1985)
    [1656, 130,   170,   40,    4,     0],  # Annett 1978
    [303,  37,    45,    15,    0,     0],  # Carter-Saltzmann 1980
    [315,  68,    57,    16,    0,     0],  # Coren & Porac 1980
    [848,  211,   325,   150,   30,   22],  # McGee & Cozad 1980
    [58,   9,     14,    5,     0,     0],  # McManus 1985 ICM1
    [134,  15,    17,    9,     1,     0],  # McManus 1985 ICM2prop
    [74,   4,     6,     2,     0,     0],  # McManus 1985 ICM2mat 
    [86,   4,     8,     1,     0,     0],  # McManus 1985 ICM2pat
    [1729, 173,   281,   63,    8,     3]   # Leiber & Axelrod 1981
]
Laland = np.array(Laland)

In [371]:
# Observed frequancy of P(offsprin|parents mating) in a given sample D:

def observed_T(D):
    
    RxR = D[:,0:2].sum()
    RxL = D[:,2:4].sum()
    LxL = D[:,4:6].sum()
    
    D_obs = pd.DataFrame( data = [
        #   R                     L 
        [D[:,0].sum()/RxR , D[:,1].sum()/RxR], #RxR
        [D[:,2].sum()/RxL , D[:,3].sum()/RxL], #RxL
        [D[:,4].sum()/LxL , D[:,5].sum()/LxL], #LxL
        ], index = ['RxR', 'RxL', 'LxL'],  columns = ['Obs_R', 'Obs_L'])
    
    return D_obs

In [372]:
observed_T(Laland)

,Obs_R,Obs_L
RxR,0.914963,0.085037
RxL,0.778800,0.221200
LxL,0.618705,0.381295


In [374]:
observed(Laland)

(       Total sample
 Left       0.072193
 Right      0.927807,
         Obs_R     Obs_L
 RxR  0.914963  0.085037
 RxL  0.778800  0.221200
 LxL  0.618705  0.381295)

In [380]:
observed(Laland)

(       Total sample
 Left       0.927807
 Right      0.072193,
         Obs_R     Obs_L
 RxR  0.914963  0.085037
 RxL  0.778800  0.221200
 LxL  0.618705  0.381295)

In [264]:
θ_1995 = np.array([0.267, 0.148, 0.012])
θ_19952d = np.array([0.277, 0.138, 0])

In [305]:
expected(0.277, 0.138, 0)

P(DR) =  0.8825966850828729 
P(DL) = 0.11740331491712708


(None,
      Ex_Right  Ex_Left
 RxR     0.915    0.085
 RxL     0.777    0.223
 LxL     0.639    0.361)

### Laland new

In [ ]:
θ_Laland_new =  np.array([0.320, 0.157, 0.000]) # bayseian inference
θ_19952d_new = np.array([0.321, 0.156, 0]) # bayseian inference

In [306]:
expected(0.321, 0.156, 0)

P(DR) =  0.9665697674418606 
P(DL) = 0.03343023255813937


(None,
      Ex_Right  Ex_Left
 RxR     0.977    0.023
 RxL     0.821    0.179
 LxL     0.665    0.335)

### Nurhayu2022

In [313]:
#    R|RxR  L|RxR  R|RxL  L|RxL  R|LxL  L|LxL

Nurhayu = [[521,   73,    61,    26,     1,    1]] # Nurhayu 2020 (Total sample parents-offspring frequency)

Nurhayu = np.array(Nurhayu)

In [314]:
observed(Nurhayu)

(       Total sample
 Left       0.093216
 Right      0.906784,
         Obs_R     Obs_L
 RxR  0.877104  0.122896
 RxL  0.701149  0.298851
 LxL  0.500000  0.500000)

In [309]:
θ_Nurhayu3D = np.array([0.096,0.393,0.043])
θ_Nurhayu2D = np.array([0.266,0.193,0])

In [317]:
validate(θ_Nurhayu3D)

True

In [350]:
expected(0.096,0.393,0.043)

P(DR) =  0.9629396044296474 
P(DL) = 0.03706039557035257


(None,
      Ex_Right  Ex_Left
 RxR     0.989    0.011
 RxL     0.639    0.361
 LxL     0.203    0.797)

In [354]:
expected(0.204, 0.265, 0.041)

P(DR) =  0.9433640844045328 
P(DL) = 0.05663591559546721


(None,
      Ex_Right  Ex_Left
 RxR     0.969    0.031
 RxL     0.745    0.255
 LxL     0.439    0.561)

In [349]:
validate(θ_Nurhayu2D)

True

In [320]:
expected(0.266,0.193,0)

P(DR) =  0.9332247557003257 
P(DL) = 0.0667752442996743


(None,
      Ex_Right  Ex_Left
 RxR     0.959    0.041
 RxL     0.766    0.234
 LxL     0.573    0.427)